In [1]:
import glob
import os

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import seaborn as sns

import scanpy.api as sc

import holoviews as hv
# hv.archive.auto()

%matplotlib inline

/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
! aws s3 ls s3://olgabot-maca/facs/sourmash_compare/ > aws_s3_ls_tabula_muris_compare.txt

In [3]:
aws_s3_ls = pd.read_table('aws_s3_ls_tabula_muris_compare.txt', 
                          delim_whitespace=True, header=None, 
                          names=['date', 'time', 'bytes', 'basename'])
print(aws_s3_ls.shape)
aws_s3_ls.head()

(722, 4)


,date,time,bytes,basename
0,2018-08-09,13:58:43,9903207,trim=false_scaled=1000_dna_ksize=12.csv
1,2018-08-09,18:11:11,10065408,trim=false_scaled=1000_dna_ksize=21.csv
2,2018-08-09,14:07:36,10092988,trim=false_scaled=1000_dna_ksize=24.csv
3,2018-08-09,17:08:12,10249183,trim=false_scaled=1000_dna_ksize=27.csv
4,2018-08-09,16:14:15,9957529,trim=false_scaled=1000_dna_ksize=33.csv


In [4]:
prefix = 's3://olgabot-maca/facs/sourmash_compare/'

In [ ]:

def extract_cell_metadata(columns):
    cell_metadata = pd.Series(columns).str.split('|', expand=True)
#     cell_metadata.index = columns
    cell_metadata.columns = ['cell_id', 'tissue', 
                             'subtissue', 'cell_ontology_class', 
                             'free_annotation']
    cell_metadata = cell_metadata.set_index('cell_id')
    cell_metadata.index.name = 'cell_id'
    cell_metadata = cell_metadata.applymap(lambda x: x.split(':')[-1])
    print('\tcell_metadata.shape', cell_metadata.shape)
#     cell_metadata = cell_metadata.sort_index()
#     cell_metadata.head()
    return cell_metadata



dfs = {}

metadatas = []

def to_key_value_pair(attribute):
    if len(attribute) > 1:
        return attribute[0], attribute[1] 
    else:
        return 'comparison_sequence', attribute[0]



def extract_experiment_metadata(basename):
    key = basename.split('.csv')[0]
    split = key.split('_')
    attributes = [x.split('=') for x in split]
    attributes = dict(to_key_value_pair(x) for x in attributes)
    return key, attributes


experiment_metadatas = {}

for basename in aws_s3_ls['basename']:
    csv = f'{prefix}{basename}'
    df = pd.read_csv(csv)
    print(f'{basename}\tdf.shape: {df.shape}')
    
    key, attributes = extract_experiment_metadata(basename)
    experiment_metadatas[key] = attributes

    metadata = extract_cell_metadata(df.columns)
    df.index = df.columns = metadata.index
        
    dfs[key] = df
    metadatas.append(metadata)



trim=false_scaled=1000_dna_ksize=12.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=21.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=24.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=27.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=33.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=36.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=39.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=42.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=45.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=54.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)
trim=false_scaled=1000_dna_ksize=57.csv	df.shape: (718, 718)
	cell_metadata.shape (718, 4)

trim=false_scaled=1500_dna_ksize=9.csv	df.shape: (919, 919)
	cell_metadata.shape (919, 4)
trim=false_scaled=1600_dna_ksize=12.csv	df.shape: (998, 998)
	cell_metadata.shape (998, 4)
trim=false_scaled=1600_dna_ksize=21.csv	df.shape: (998, 998)
	cell_metadata.shape (998, 4)
trim=false_scaled=1600_dna_ksize=24.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
trim=false_scaled=1600_dna_ksize=27.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
trim=false_scaled=1600_dna_ksize=3.csv	df.shape: (998, 998)
	cell_metadata.shape (998, 4)
trim=false_scaled=1600_dna_ksize=30.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
trim=false_scaled=1600_dna_ksize=33.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
trim=false_scaled=1600_dna_ksize=36.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
trim=false_scaled=1600_dna_ksize=39.csv	df.shape: (998, 998)
	cell_metadata.shape (998, 4)
trim=false_scaled=1600_dna_ksize=45.csv	df.shape: (738, 738)
	cell_metadata.shape (738, 4)
t

trim=false_scaled=2500_dna_ksize=54.csv	df.shape: (926, 926)
	cell_metadata.shape (926, 4)
trim=false_scaled=2500_dna_ksize=57.csv	df.shape: (926, 926)
	cell_metadata.shape (926, 4)
trim=false_scaled=2500_dna_ksize=6.csv	df.shape: (926, 926)
	cell_metadata.shape (926, 4)
trim=false_scaled=2500_dna_ksize=60.csv	df.shape: (926, 926)
	cell_metadata.shape (926, 4)
trim=false_scaled=2500_dna_ksize=9.csv	df.shape: (926, 926)
	cell_metadata.shape (926, 4)
trim=false_scaled=3000_dna_ksize=12.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
trim=false_scaled=3000_dna_ksize=15.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
trim=false_scaled=3000_dna_ksize=18.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
trim=false_scaled=3000_dna_ksize=21.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
trim=false_scaled=3000_dna_ksize=24.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
trim=false_scaled=3000_dna_ksize=27.csv	df.shape: (741, 741)
	cell_metadata.shape (741, 4)
t

trim=false_scaled=4500_dna_ksize=6.csv	df.shape: (732, 732)
	cell_metadata.shape (732, 4)
trim=false_scaled=4500_dna_ksize=60.csv	df.shape: (997, 997)
	cell_metadata.shape (997, 4)
trim=false_scaled=4500_dna_ksize=9.csv	df.shape: (732, 732)
	cell_metadata.shape (732, 4)
trim=false_scaled=5000_dna_ksize=12.csv	df.shape: (996, 996)
	cell_metadata.shape (996, 4)
trim=false_scaled=5000_dna_ksize=15.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=18.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=21.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=24.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=3.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=30.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=5000_dna_ksize=33.csv	df.shape: (996, 996)
	cell_metadata.shape (996, 4)
tr

trim=false_scaled=900_dna_ksize=60.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=false_scaled=900_dna_ksize=9.csv	df.shape: (749, 749)
	cell_metadata.shape (749, 4)
trim=true_scaled=1000_dna_ksize=12.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=15.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=18.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=21.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=24.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=30.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=33.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=36.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_scaled=1000_dna_ksize=39.csv	df.shape: (748, 748)
	cell_metadata.shape (748, 4)
trim=true_s

trim=true_scaled=1500_dna_ksize=54.csv	df.shape: (759, 759)
	cell_metadata.shape (759, 4)
trim=true_scaled=1500_dna_ksize=57.csv	df.shape: (997, 997)
	cell_metadata.shape (997, 4)
trim=true_scaled=1500_dna_ksize=6.csv	df.shape: (759, 759)
	cell_metadata.shape (759, 4)
trim=true_scaled=1500_dna_ksize=9.csv	df.shape: (997, 997)
	cell_metadata.shape (997, 4)
trim=true_scaled=1600_dna_ksize=12.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_scaled=1600_dna_ksize=15.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_scaled=1600_dna_ksize=18.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_scaled=1600_dna_ksize=21.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_scaled=1600_dna_ksize=24.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_scaled=1600_dna_ksize=27.csv	df.shape: (999, 999)
	cell_metadata.shape (999, 4)
trim=true_scaled=1600_dna_ksize=3.csv	df.shape: (767, 767)
	cell_metadata.shape (767, 4)
trim=true_sca

trim=true_scaled=2500_dna_ksize=54.csv	df.shape: (733, 733)
	cell_metadata.shape (733, 4)
trim=true_scaled=2500_dna_ksize=57.csv	df.shape: (994, 994)
	cell_metadata.shape (994, 4)
trim=true_scaled=2500_dna_ksize=6.csv	df.shape: (733, 733)
	cell_metadata.shape (733, 4)
trim=true_scaled=2500_dna_ksize=60.csv	df.shape: (994, 994)
	cell_metadata.shape (994, 4)
trim=true_scaled=2500_dna_ksize=9.csv	df.shape: (733, 733)
	cell_metadata.shape (733, 4)
trim=true_scaled=3000_dna_ksize=15.csv	df.shape: (999, 999)
	cell_metadata.shape (999, 4)
trim=true_scaled=3000_dna_ksize=18.csv	df.shape: (751, 751)
	cell_metadata.shape (751, 4)
trim=true_scaled=3000_dna_ksize=21.csv	df.shape: (751, 751)
	cell_metadata.shape (751, 4)


In [ ]:
cell_metadata = pd.concat(metadatas)
print(cell_metadata.shape)
cell_metadata = cell_metadata.loc[~cell_metadata.index.duplicated()]
print(cell_metadata.shape)
cell_metadata.head()

In [ ]:
experiment_metadata = pd.DataFrame(experiment_metadatas).T
print(experiment_metadata.shape)
experiment_metadata.head()

In [ ]:
df.head()

In [ ]:
jaccard_indices = xr.Dataset(dfs)
jaccard_indices

In [17]:
jaccard_indices

NameError: name 'ds' is not defined